In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [13]:
path = r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Datos_Proyecto"

df_observations = pd.read_csv(r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Datos_Proyecto\observations.csv")

**CHECK FOR NA VALUES ON DATA**

In [ ]:
for csv in os.listdir(path):
    df = pd.read_csv(os.path.join(path, csv)) 
    print(f"{csv}:\n{df.isna().sum()}\n")

VISUALIZACION DE VALORES NULOS

In [ ]:
# 2. Resumen de los datos nulos
print("Resumen de valores nulos por columna:")
print(df_observations.isnull().sum())

# 3. Total de nulos
total_nulos = df_observations.isnull().sum().sum()
print(f"\nTotal de valores nulos en el DataFrame: {total_nulos}")

# 4. Porcentaje de datos nulos por columna
porcentaje_nulos = (df_observations.isnull().sum() / len(df_observations)) * 100
print("\nPorcentaje de valores nulos por columna:")
print(porcentaje_nulos)

In [ ]:
# Calcular la cantidad de valores nulos por columna
null_counts = df_observations.isnull().sum()

# Calcular el total de registros por columna
total_counts = df_observations.shape[0]

# Crear una nueva DataFrame con los datos para graficar
data = pd.DataFrame({
    'No Nulos': total_counts - null_counts.values,
    'Nulos': null_counts.values
}, index=null_counts.index)

# Configurar el tamaño de la figura
plt.figure(figsize=(15, 10))

# Crear el gráfico de barras apiladas (gráfico de columnas agrupadas)
data.plot(kind='bar', stacked=False, color=['green', 'red'])

# Títulos y etiquetas
plt.title("Total de Registros vs Valores Nulos por Columna")
plt.xlabel("Columnas")
plt.ylabel("Cantidad de Registros")
plt.xticks(rotation=90)  # Rotar las etiquetas del eje x si es necesario

# Agregar leyenda
plt.legend(["No Nulos", "Nulos"])

# Mostrar el gráfico
plt.show()

In [ ]:
# Estadísticas descriptivas de las columnas con nulos
cols_nulos = ['precipitation', 'temp_max', 'temp_min', 'wind']
print(df_observations[cols_nulos].describe())

# Histograma de cada columna
for col in cols_nulos:
    plt.figure(figsize=(8, 5))
    sns.histplot(df_observations[col], kde=True, bins=30, color='blue')
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

**ANALISIS DE DATOS CON NULOS**

In [ ]:
data = pd.read_csv(r"E:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations.csv")

data_cleaned = data.dropna()

numerical_data = data_cleaned.select_dtypes(include=['float64', 'int64'])

correlation_matrix = numerical_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlaciones')
plt.show()


**VISUALIZAR LOS DATOS QUE RESALTAN EN EL GRÁFICO DE CORRELACIONES**

WEATHER_ID

In [ ]:
# Boxplot de weather_id vs precipitación
plt.figure(figsize=(10, 6))
sns.boxplot(x='weather_id', y='precipitation', hue='weather_id', data=data_cleaned, palette='Blues', legend=False)
plt.title('Distribución de Precipitación según Weather ID')
plt.xlabel('Weather ID')
plt.ylabel('Precipitación')
plt.show()

# Boxplot de weather_id vs viento
plt.figure(figsize=(10, 6))
sns.boxplot(x='weather_id', y='wind', data=data_cleaned, palette='Greens')
plt.title('Distribución de Viento según Weather ID')
plt.xlabel('Weather ID')
plt.ylabel('Viento')
plt.show()

In [ ]:
# Boxplot de weather_id vs viento
plt.figure(figsize=(10, 6))
sns.boxplot(x='cloudiness_id', y='solar_radiation', data=data_cleaned, palette='Reds')
plt.title('Distribución de Radiación Solar según Cloudiness ID')
plt.xlabel('Cloudiness ID')
plt.ylabel('Radiación Solar')
plt.show()

**METODOS DE IMPUTACIÓN DE VALORES NULOS**

con los valores previos y posteriores del valor nulo

In [5]:
df = pd.read_csv(r"E:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations.csv")
dates = pd.read_csv(r"E:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\dates.csv")  # Archivo con las fechas

# Unir la fecha al DataFrame principal
df = df.merge(dates, on='date_id', how='left')

# Convertir la fecha a formato datetime y extrae
df['date'] = pd.to_datetime(df['date'])

# ordenar por 'date'
df = df.sort_values(by='date')

# Calcular las medias por estación y año para las columnas con valores nulos
columns_to_impute = ['precipitation', 'temp_max', 'temp_min', 'wind']

for col in columns_to_impute:
    df[col] = df[col].interpolate(method='linear', limit_direction='both')

df.to_csv(r"E:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv", index=False)



**ANALISIS DE DATOS SIN NULOS**

In [ ]:
full_data = pd.read_csv(r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv")

numerical_data = full_data.select_dtypes(include=['float64', 'int64'])

correlation_matrix = numerical_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlaciones')
plt.show()

In [ ]:
# Estadísticas descriptivas de las columnas que contenian nulos
cols_nulos = ['precipitation', 'temp_max', 'temp_min', 'wind']
print(full_data[cols_nulos].describe())

# Histograma de cada columna
for col in cols_nulos:
    plt.figure(figsize=(8, 5))
    sns.histplot(full_data[col], kde=True, bins=30, color='blue')
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

**VISUALIZAR LOS DATOS QUE RESALTAN EN EL GRÁFICO DE CORRELACIONES**

WEATHER ID

In [ ]:
# Boxplot de weather_id vs precipitación
plt.figure(figsize=(10, 6))
sns.boxplot(x='weather_id', y='precipitation', hue='weather_id', data=full_data, palette='Blues', legend=False)
plt.title('Distribución de Precipitación según Weather ID')
plt.xlabel('Weather ID')
plt.ylabel('Precipitación')
plt.show()

PRECIPITATION

In [ ]:
# Boxplot de weather_id vs viento
plt.figure(figsize=(10, 6))
sns.boxplot(x='weather_id', y='wind', data=full_data, palette='Greens')
plt.title('Distribución de Viento según Weather ID')
plt.xlabel('Weather ID')
plt.ylabel('Viento')
plt.show()

WEATHER ID vs WIND

In [ ]:
# Boxplot de weather_id vs viento
plt.figure(figsize=(10, 6))
sns.boxplot(x='cloudiness_id', y='solar_radiation', data=full_data, palette='Reds')
plt.title('Distribución de Radiación Solar según Cloudiness ID')
plt.xlabel('Cloudiness ID')
plt.ylabel('Radiación Solar')
plt.show()

**CARGAR LOS DATOS SQL Y CREAR LOS GRAFICOS**

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

conn = sqlite3.connect(r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Entregables\Code\weather.db")

# Cargar cada tabla en un DataFrame de pandas
df_cloudiness = pd.read_sql_query("SELECT * FROM cloudiness", conn)
df_dates = pd.read_sql_query("SELECT * FROM dates", conn)
df_observations = pd.read_sql_query("SELECT * FROM observations_full", conn)
df_weather = pd.read_sql_query("SELECT * FROM weather", conn)
df_seasons = pd.read_sql_query("SELECT * FROM seasons", conn)

# Cerrar la conexión cuando hayas terminado
conn.close()

# Unir las tablas usando merge
df = df_observations \
    .merge(df_dates, on='date_id') \
    .merge(df_weather, on='weather_id') \
    .merge(df_cloudiness, on='cloudiness_id') \
    .merge(df_seasons, on='estacion_id')


DISTRIBUCIÓN DE LAS TEMPERATURAS MAXIMAS

In [ ]:
# Graficar un histograma de temperatura máxima
plt.figure(figsize=(10, 6))
sns.histplot(df['temp_max'].dropna(), bins=30, kde=True)
plt.title("Distribución de Temperatura Máxima")
plt.xlabel("Temperatura Máxima (°C)")
plt.ylabel("Frecuencia")
plt.show()

TEMPERATURA MAX/MIN POR FECHA

In [ ]:
# Convertir la columna de fechas a tipo datetime
df['date'] = pd.to_datetime(df['date'])

# Graficar temperatura máxima y mínima a lo largo del tiempo
plt.figure(figsize=(14, 8))
sns.lineplot(x='date', y='temp_max', data=df, label='Temp Max')
sns.lineplot(x='date', y='temp_min', data=df, label='Temp Min')
plt.title("Temperatura Máxima y Mínima a lo Largo del Tiempo")
plt.xlabel("Fecha")
plt.ylabel("Temperatura (°C)")
plt.legend()
plt.show()

HUMEDAD POR CONDICION CLIMATICA

In [ ]:
# Calcular la humedad promedio por tipo de clima
df_weather_humidity = df.groupby('weather')['humidity'].mean().reset_index()

# Graficar la humedad promedio por tipo de clima
plt.figure(figsize=(10, 6))
sns.barplot(x='humidity', y='weather', hue='weather', data=df_weather_humidity, palette="viridis", legend=False)
plt.title("Humedad Promedio por Condición Climática")
plt.xlabel("Humedad (%)")
plt.ylabel("Condición Climática")
plt.show()

TEMPERATURA MÁXIMA POR TIPO DE TIEMPO

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='weather', y='temp_max', hue='weather', data=df, palette="Set3", legend=False)
plt.title("Distribución de Temperatura Máxima por Condición Climática")
plt.xlabel("Condición Climática")
plt.ylabel("Temperatura Máxima (°C)")
plt.show()

VIENTO vs PRESIÓN

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='wind', y='pressure', hue='cloudiness', data=df, palette="bright")
plt.title("Viento vs. Presión según Nubosidad")
plt.xlabel("Velocidad del Viento (m/s)")
plt.ylabel("Presión (hPa)")
plt.legend(title="Nubosidad", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

PRECIPITACION POR VIENTO Y TIPO DE TIEMPO

In [ ]:
# Graficar
plt.figure(figsize=(12, 8))
sns.scatterplot(x='wind', y='precipitation', hue='weather', data=df, palette="Set2", s=50)
plt.title("Precipitación vs. Viento por Weather")
plt.xlabel("Velocidad del Viento (m/s)")
plt.ylabel("Precipitación (mm)")
plt.legend(title='Tipo de Tiempo', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()


PRECIPITACIÓN POR TIPO DE TIEMPO

In [ ]:
# Graficar
plt.figure(figsize=(12, 6))
sns.barplot(x='weather', y='precipitation', hue='weather', data=df, palette="viridis", legend=False)
plt.title("Precipitación Promedio por Weather")
plt.xlabel("Tipo de Tiempo")
plt.ylabel("Precipitación Promedio (mm)")
plt.xticks(rotation=45)
plt.show()

TEMPERATURA PROMEDIO POR MES

In [ ]:
# CONVERTIR date A datetime
df['date'] = pd.to_datetime(df['date'])

# EXTRAER MES Y AÑO
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# AGRUPAR POR MES Y CALCULAR LA TEMPERATURA PROMEDIO
df_temp_by_month = df.groupby('month').agg({'temp_max': 'mean', 'temp_min': 'mean'}).reset_index()

# MOSTRAR GRAFICO
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_temp_by_month, x='month', y='temp_max', marker='o', label='Temp Max', color='red')
sns.lineplot(data=df_temp_by_month, x='month', y='temp_min', marker='o', label='Temp Min', color='blue')
plt.title("Temperatura Promedio por Mes")
plt.xlabel("Mes")
plt.ylabel("Temperatura (°C)")
plt.xticks(ticks=df_temp_by_month['month'], labels=["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"])
plt.legend()
plt.show()


TEMPERATURA PROMEDIO POR AÑO

In [ ]:
# CONVERTIR date A datetime
df['date'] = pd.to_datetime(df['date'])

# EXTRAER MES Y AÑO
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# AGRUPAR POR MES Y CALCULAR LA TEMPERATURA PROMEDIO
df_temp_by_month = df.groupby('year').agg({'temp_max': 'mean', 'temp_min': 'mean'}).reset_index()

# MOSTRAR GRAFICO
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_temp_by_month, x='year', y='temp_max', marker='o', label='Temp Max', color='red')
sns.lineplot(data=df_temp_by_month, x='year', y='temp_min', marker='o', label='Temp Min', color='blue')
plt.title("Temperatura Promedio por Año")
plt.xlabel("Year")
plt.ylabel("Temperatura (°C)")
plt.legend()
plt.show()


VARIABILIDAD DE LA VISIBILIDAD POR CONDICIÓN CLIMÁTICA

In [ ]:
# Grafica de la visibilidad por condición climática
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='weather', y='visibility', hue='weather_id', palette="viridis", legend=False)
plt.title('Variabilidad de la Visibilidad por Condición Climática')
plt.xlabel('Condición Climática')
plt.ylabel('Visibilidad')
plt.grid(axis='y')
plt.show()


TEMPERATURA MÁXIMA Y MÍNIMA PROMEDIO POR ESTACIÓN DEL AÑO

In [ ]:
# Temperatura promedio por estación
temp_seasonal_avg = df.groupby('estacion')[['temp_max', 'temp_min']].mean()

# Gráfica de barras para temperatura máxima y mínima por estación
plt.figure(figsize=(10, 6))
temp_seasonal_avg.plot(kind='bar', color=['#ff8f1e','#47bfff'])
plt.title('Temperatura Máxima y Mínima Promedio por Estación del Año')
plt.xlabel('Estación')
plt.xticks(ticks=range(4), labels=['Invierno', 'Primavera', 'Verano', 'Otoño'], rotation=0)
plt.ylabel('Temperatura Promedio')
plt.legend(['Temp Max', 'Temp Min'])
plt.grid(axis='y')
plt.show()


# MODELO all features

## SARIMA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv(r'C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv')

df['date'] = pd.to_datetime(df['date'])

df.set_index('date',inplace=True)

df = df.asfreq('D')

In [ ]:
import os
import pandas as pd
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAX
import concurrent.futures

# Lista de variables para las que queremos hacer predicciones
variables = ['precipitation', 'temp_max', 'temp_min', 'wind', 'humidity', 'pressure', 'solar_radiation', 'visibility', 'weather_id', 'cloudiness_id']

# Ruta donde se guardarán los modelos entrenados
models_path = "sarima_models"
os.makedirs(models_path, exist_ok=True)  # Crear el directorio si no existe

# Función para ajustar y guardar el modelo si no existe, o cargarlo si ya está guardado
def fit_or_load_model(variable_name):
    model_filename = os.path.join(models_path, f"{variable_name}_sarima.pkl")
    
    if os.path.exists(model_filename):
        # Si el modelo ya existe, cargarlo
        sarima_model = joblib.load(model_filename)
        print(f"Modelo SARIMA para {variable_name} cargado desde {model_filename}")
    else:
        # Si el modelo no existe, ajustarlo y guardarlo
        model = SARIMAX(df[variable_name],
                        order=(1, 1, 1),
                        seasonal_order=(1, 1, 1, 12),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        
        sarima_model = model.fit(disp=False, method='powell')
        joblib.dump(sarima_model, model_filename, compress=4)
        print(f"Modelo SARIMA para {variable_name} ajustado y guardado en {model_filename}")
    
    return sarima_model

# Función para hacer predicciones usando un modelo ajustado o cargado
def forecast_with_model(variable_name):
    sarima_model = fit_or_load_model(variable_name)
    forecast = sarima_model.get_forecast(steps=31)
    forecast_mean = forecast.predicted_mean
    return forecast_mean

# Ejecutar las predicciones en paralelo usando ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    forecasts = list(executor.map(forecast_with_model, variables))

# Almacenar los resultados en un DataFrame para facilidad de visualización
predictions_df = pd.DataFrame({variables[i]: forecasts[i] for i in range(len(variables))})

predictions_df.to_csv(r'C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Entregables\Code\predictions.csv')


## Random Forest Regressor

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# Load data
df = pd.read_csv(r"D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv")

# Extract time-based features
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Drop the original date column
df = df.drop(columns=['date'])

# Select features and targets
features = ['precipitation', 'temp_max','temp_min', 'wind', 'humidity', 'pressure', 
            'solar_radiation', 'visibility', 'weather_id', 'estacion_id', 'cloudiness_id', 
            'year', 'month', 'day']

targets = ['precipitation', 'temp_max','temp_min', 'wind', 'humidity', 'pressure']

for target in targets:

    features_clean = [feature for feature in features if feature != target]

    # Separate features and target
    X = df[features_clean]
    y = df[target]

    # Split data into train and test sets (using time-based split for time series)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Train a Random Forest model for prediction
    model = RandomForestRegressor(n_estimators=300, max_depth=15, min_samples_split=5, min_samples_leaf=4, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"\nModelo {target}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # Save the model using joblib
    model_path = fr'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\SVM\features_prediction\{target}_rfr_model.pkl'
    joblib.dump(model, model_path)

    print(f"Modelo {target} guardado en: {model_path} (intro)")

## PROPHET

Entrenar el modelo y hacer predicciones

In [ ]:
import joblib
import pandas as pd
from plotly import graph_objs as go
from prophet import Prophet
from prophet.plot import plot_plotly

df = pd.read_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv')

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['date'], y=df['humidity'], name='humidity',line_color='red'))
# fig.layout.update(title_text='Time Series data with Rangeslider',xaxis_rangeslider_visible=True)
# fig.show()

targets = ['precipitation','temp_max','temp_min','wind','humidity','pressure','solar_radiation','visibility','cloudiness_id']

for target in targets:

    X = df[['date',target]]
    y = df[target]

    train_df = pd.DataFrame()
    train_df['ds'] = pd.to_datetime(X['date'])
    train_df['y']=y
    train_df.head(2)

    model = Prophet()
    model.fit(train_df)
    joblib.dump(model,fr"D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\Prophet\models\prophet_{target}.pkl")


    future = model.make_future_dataframe(periods=365)
    forecast = model.predict(future)
    forecast.to_csv(fr"D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\Prophet\Predictions_2025\predictions_{target}.csv")

    # fig1 = plot_plotly(model, forecast)
    # fig1.show()

    # #plot component wise forecast
    # fig2 = model.plot_components(forecast)
    # fig2.show()


## XGBOOSTING

In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# Load data
df = pd.read_csv(r"D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv")

# Extract time-based features
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Drop the original date column
df = df.drop(columns=['date'])

# Select features and targets
features = ['year', 'month', 'day']
X = df[features]

targets = ['precipitation', 'temp_max','temp_min', 'wind', 'humidity', 'pressure','solar_radiation','visibility','cloudiness_id']

# Hyperparameter grid
param_grid = {
    'n_estimators': [500, 1000],
    'max_depth': [10, 20, 50],
    'min_child_weight': [5, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.9, 1.0]
}

for target in targets:
    # Separate features and target
    y = df[target]

    # Split data into train and test sets (using time-based split for time series)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize XGBoost model
    model_xgb = XGBRegressor(random_state=42)

    # Perform GridSearchCV to find the best hyperparameters
    grid_search = GridSearchCV(estimator=model_xgb, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)

    # Fit the grid search model
    grid_search.fit(X_train, y_train)

    # Best parameters from the grid search
    best_params = grid_search.best_params_
    print(f"\nMejores parámetros para el modelo {target}: {best_params}")

    # Refit the best model
    best_model = grid_search.best_estimator_

    # Evaluate the best model on the test data
    y_pred_xgb = best_model.predict(X_test)
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

    print(f"\nModelo {target} - XGBoost (Con mejores parámetros)")
    print(f"Mean Absolute Error (MAE): {mae_xgb}")
    print(f"Root Mean Squared Error (RMSE): {rmse_xgb}")

    # Save the best XGBoost model
    model_xgb_path = fr'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\XGBoost\{target}_xgb_model_best.pkl'
    joblib.dump(best_model, model_xgb_path)
    print(f"Modelo {target} guardado en: {model_xgb_path} (XGB - Mejores parámetros)")


Fitting 3 folds for each of 144 candidates, totalling 432 fits

Mejores parámetros para el modelo precipitation: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 10, 'min_child_weight': 8, 'n_estimators': 500, 'subsample': 0.7}

Modelo precipitation - XGBoost (Con mejores parámetros)
Mean Absolute Error (MAE): 13.942166633159891
Root Mean Squared Error (RMSE): 16.305101710767545
Modelo precipitation guardado en: D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\XGBoost\precipitation_xgb_model_best.pkl (XGB - Mejores parámetros)
Fitting 3 folds for each of 144 candidates, totalling 432 fits

Mejores parámetros para el modelo temp_max: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 10, 'min_child_weight': 8, 'n_estimators': 500, 'subsample': 0.7}

Modelo temp_max - XGBoost (Con mejores parámetros)
Mean Absolute Error (MAE): 9.217679092750755
Root Mean Squared Error (RMSE): 10.911769981897358
Modelo temp_max guardado en: D:\STUCOM\Master_IABD\Projec

Juntar las predicciones de todos los targets en un solo data frame

In [13]:
import os,re
import pandas as pd

df_predictions = pd.DataFrame()

path = r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\Prophet\Predictions_2025'

_, _, files = next(os.walk(path))
    
for file in files:

    df = pd.read_csv(os.path.join(path,file))

    if 'cloudiness' in file:
        df['yhat'] = df['yhat'].round(0).astype(int)

    
    df_predictions[re.search(r'predictions_(.*?).csv', file).group(1)] = df['yhat']

df_predictions.insert(0,'date',df['ds'])

df_predictions = df_predictions.drop(df_predictions.index[:25000]).reset_index(drop=True)


df_predictions.to_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\predictions\predictions_Prophet.csv')



# MODELO weather_id

# SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Load dataset
data = pd.read_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv')

# Process date and add additional time-based features
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

# Select features and target
features = ['year', 'month', 'day', 'precipitation', 'temp_max', 'temp_min', 'wind', 'humidity', 'pressure', 'solar_radiation', 'visibility', 'cloudiness_id']
target = 'weather_id'

# Split data into train and test sets
X = data[features]
y = data[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# model = SVC(kernel='rbf', C=0.1, gamma='auto', class_weight='balanced', probability=True) Model 3
model = SVC(kernel='rbf', C=0.1, gamma='auto', probability=True) # Model 4


model.fit(X_train_res, y_train_res)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# ROC AUC (for multi-class)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled), multi_class='ovr')
print(f"ROC AUC: {roc_auc}")

# Save the model using joblib
joblib.dump(model, r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\SVM\weather_id_svm_model_v4.pkl')


# RANDOM FOREST CLASSIFIER

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
import joblib

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

# Load dataset
data = pd.read_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\observations_full.csv')

# Process date and add additional time-based features
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

# Select features and target
features = ['year', 'month', 'day', 'precipitation', 'temp_max', 'temp_min', 'wind', 'humidity', 'pressure', 'solar_radiation', 'visibility', 'cloudiness_id']
target = 'weather_id'

# Split data into train and test sets
X = data[features]
y = data[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# RANDOM FOREST

model_rf = RandomForestClassifier(class_weight='balanced', random_state=42)
model_rf.fit(X_train_res, y_train_res)
y_pred_rf = model_rf.predict(X_test_scaled)

print("Random Forest Classifier - Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

joblib.dump(model_rf,r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\RandomForest\weather_id_RF.pkl')


## REALIZAR LAS PREDICCIONES

Usando los valores de los targets

In [ ]:
import pandas as pd
import joblib

df_predictions = pd.read_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\predictions\predictions_Prophet.csv')
df_predictions.drop('Unnamed: 0', axis=1, inplace=True)
df_predictions_copy = df_predictions.copy()

df_predictions_copy['date'] = pd.to_datetime(df_predictions_copy['date'])
df_predictions_copy['year'] = df_predictions_copy['date'].dt.year
df_predictions_copy['month'] = df_predictions_copy['date'].dt.month
df_predictions_copy['day'] = df_predictions_copy['date'].dt.day
df_predictions_copy.drop('date', axis=1, inplace=True)

model = joblib.load(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Entregables\Code\RandomForest\weather_id_RF.pkl')

predictions = model.predict(df_predictions_copy)

df_predictions['weather_id'] = predictions

df_predictions.to_csv(r'D:\STUCOM\Master_IABD\Projecte3_Meteorologia\Datos_Proyecto\predictions\predictions_Prophet_full.csv')

VISUALIZACION DE PREDICCIONES VS VALORES REALES

In [ ]:
import matplotlib.pyplot as plt

# Graficar las predicciones vs los valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)  # Línea de igualdad
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Valores Reales vs Predicciones')
plt.show()


CROSS VALIDATION

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, data[features], data[target], cv=5, scoring='neg_mean_squared_error')
print("Cross-validation RMSE scores:", np.sqrt(-scores))  # Convertir de negativo a positivo


In [ ]:
import pandas as pd
import joblib
from datetime import datetime

# Cargar el modelo entrenado
target = 'precipitation'  # O el target de tu interés
model = joblib.load(f'E:\\STUCOM\\Master_IABD\\Projecte3_Meteorologia\\Entregables\\Code\\RandomForest_Models\\{target}_model.pkl')

# Crear un DataFrame con las fechas de interés
fechas_interes = ['2025-01-10', '2025-01-11', '2025-01-12']  # Agrega las fechas para las que deseas predecir
fechas_interes = pd.to_datetime(fechas_interes)

# Crear las características correspondientes para las fechas de interés
data_fechas = pd.DataFrame({
    'date': fechas_interes,
    'year': fechas_interes.year,
    'month': fechas_interes.month,
    'day': fechas_interes.day,
    'precipitation': [0] * len(fechas_interes),  # La precipitación no importa en este caso
    'temp_max': [0] * len(fechas_interes),  # O puedes poner valores estimados si los tienes
    'temp_min': [0] * len(fechas_interes),  # Lo mismo para las otras variables
    'wind': [0] * len(fechas_interes),
    'humidity': [0] * len(fechas_interes),
    'pressure': [0] * len(fechas_interes),
    'solar_radiation': [0] * len(fechas_interes),
    'visibility': [0] * len(fechas_interes),
    'weather_id': [0] * len(fechas_interes),
    'cloudiness_id': [0] * len(fechas_interes)
})

# Realizar las predicciones
X_fechas_interes = data_fechas[features]  # Seleccionar las características
predicciones = model.predict(X_fechas_interes)

# Agregar las predicciones al DataFrame
data_fechas['predicted_precipitation'] = predicciones

# Mostrar las predicciones
print(data_fechas[['date', 'predicted_precipitation']])
